# Generate beta factor text files

In [2]:
import os, sys, shutil, subprocess, matplotlib
import pandas as pd
import numpy as np
import scipy as sp

path     = os.getcwd()
path_ds  = path+'/opt_det-seqs'  # use for normalize bend and energy
path_dp  = path+'/opt_det-pars'  # use for excess roll and twist
# --------------------------------------------------------------------------------
pathbeta = path+'/opt_beta-factors'


In [3]:
Ncirc = 150

#betafactors = ['bend_norm', 'energy_norm', 'twist_norm','radius_norm','wmin_norm',
#               'roll_excess','twist_excess',
#               'roll_diff','twist_diff','bend_diff','radius_diff','energy_diff']
betafactors = ['bend_excess']

for i in range(0, len(betafactors)):
    if not os.path.exists(pathbeta+'/'+betafactors[i]):
        os.mkdir(pathbeta+'/'+betafactors[i])

def load_optdetailed_df(filepath):
    infile = open(filepath, 'r')
    indata = infile.readlines()
    infile.close()
    indata = [i.rstrip('\n').split() for i in indata]
    for i in range(0, len(indata)):
        for j, x in enumerate(indata[i]):
            try:
                indata[i][j] = float(x)
            except ValueError:
                pass
    df = pd.DataFrame.from_records(indata[2:], columns=indata[1:2])
    df.index = np.arange(0, len(df))
    del indata
    return df


In [4]:
lst  = []
seqs = []
for filename in os.listdir(path_ds):
    lst.append(filename.split('.')[0])
for i in lst:
    i = i.replace('_bps-data', '')
    seqs.append(i)
seqs = sorted(seqs)
del lst

## Normalized bend, energy, twist

In [ ]:
par_name = {'Bend':'bend-norm', 'Energy':'energy-norm', 'Twist':'twist-norm'}
par_dir  = {'Bend':'bend_norm', 'Energy':'energy_norm', 'Twist':'twist_norm'}
for Par in ['Bend','Energy','Twist']:
    for i in range(0, len(seqs)):
        df = load_optdetailed_df(path_ds+'/'+seqs[i]+'_bps-data.txt')
        dfb = df[Par].copy()
        dfb.loc[0, Par] = dfb.loc[Ncirc, Par]
        
        # ----- New: 21 Nov 2019
        # in .pdb, b/c chain A = 1-Ncirc+1, chain B = Ncirc+2 - 2(Ncirc+1) in reverse order
        # extend list
        for j in range(0, len(df)):
            A = dfb.loc[j, Par]
            B = (2*(Ncirc+1)-j)-1
            dfb.loc[B, Par] = A
        dfb = dfb.sort_index()
        # ----- ----------------
        dfb['norm'] = (dfb[Par]-df[Par].min())/(df[Par].max() -df[Par].min())
        
        np.savetxt(path+'/'+seqs[i]+'_'+par_name[Par]+'.txt', dfb.norm.values, fmt='%8.6f')
        
        #shutil.move(path+'/'+seqs[i]+'_'+par_name[Par]+'.txt', pathbeta+'/'+par_name[Par])
        
        shutil.move(os.path.join(path, seqs[i]+'_'+par_name[Par]+'.txt'),
                    os.path.join(pathbeta+'/'+par_dir[Par], seqs[i]+'_'+par_name[Par]+'.txt'))
        del df,dfb

In [ ]:
par_name = {'Dcenter':'radius-norm', 'W-min':'wmin-norm'}
par_dir  = {'Dcenter':'radius_norm', 'W-min':'wmin_norm'}
for Par in ['Dcenter','W-min']:
    for i in range(0, len(seqs)):
        df = load_optdetailed_df(path_ds+'/'+seqs[i]+'_bps-data.txt')
        dfb = df[Par].copy()
        dfb.loc[Ncirc, Par] = dfb.loc[0, Par]
        
        # ----- New: 21 Nov 2019
        # in .pdb, b/c chain A = 1-Ncirc+1, chain B = Ncirc+2 - 2(Ncirc+1) in reverse order
        # extend list
        for j in range(0, len(df)):
            A = dfb.loc[j, Par]
            B = (2*(Ncirc+1)-j)-1
            dfb.loc[B, Par] = A
        dfb = dfb.sort_index()
        # ----- ----------------

        dfb['norm'] = (dfb[Par]-df[Par].min())/(df[Par].max() -df[Par].min())
        np.savetxt(path+'/'+seqs[i]+'_'+par_name[Par]+'.txt', dfb.norm.values, fmt='%8.6f')
        #shutil.move(path+'/'+seqs[i]+'_'+par_name[Par]+'.txt', pathbeta+'/'+par_name[Par])
        shutil.move(os.path.join(path, seqs[i]+'_'+par_name[Par]+'.txt'),
                    os.path.join(pathbeta+'/'+par_dir[Par], seqs[i]+'_'+par_name[Par]+'.txt'))
        del df,dfb

## Excess roll and twist

In [5]:
incons  = ['oring','pcirc']
ffields = ['kabsch','olson']
par     = ['Bend']
ep      = {'Bend':'bend'}

inseqs = ['col'+str(i).zfill(2) for i in range(1,10)]

for i in range(0, len(incons)):
    for j in range(0, len(ffields)):
        for k in range(0, len(par)):
            df = pd.read_csv(path_dp+'/'+incons[i]+'_'+ffields[j]+'_'+par[k], index_col=0)
            df = df.reset_index()
            for m in range(0, len(inseqs)):
                dfm = df['delta_'+inseqs[m]].copy()      
                dfm.loc[150] = 0.00
                for x in range(0, len(dfm)):
                    A = dfm.loc[x]
                    B = (2*(Ncirc+1)-x)-1
                    dfm.loc[B] = A
                dfm = dfm.sort_index()
                
                seq = inseqs[m]+'_'+incons[i]+'_std_'+ffields[j]
                np.savetxt(path+'/'+seq+'_'+ep[par[k]]+'-excess.txt', dfm.values, fmt='%10.5f')
                shutil.move(os.path.join(path, seq+'_'+ep[par[k]]+'-excess.txt'), 
                            os.path.join(pathbeta+'/'+ep[par[k]]+'_excess', seq+'_'+ep[par[k]]+'-excess.txt'))
                del dfm,seq
            del df

In [ ]:
incons  = ['oring','pcirc']
ffields = ['kabsch','olson']
par     = ['Roll','Twist']
ep      = {'Roll':'roll','Twist':'twist'}

inseqs = ['col'+str(i).zfill(2) for i in range(1,10)]

for i in range(0, len(incons)):
    for j in range(0, len(ffields)):
        for k in range(0, len(par)):
            df = pd.read_csv(path_dp+'/'+incons[i]+'_'+ffields[j]+'_'+par[k], index_col=0)
            df = df.reset_index()
            for m in range(0, len(inseqs)):
                dfm = df['delta_'+inseqs[m]].copy()      
                dfm.loc[150] = 0.00
                for x in range(0, len(dfm)):
                    A = dfm.loc[x]
                    B = (2*(Ncirc+1)-x)-1
                    dfm.loc[B] = A
                dfm = dfm.sort_index()
                
                seq = inseqs[m]+'_'+incons[i]+'_std_'+ffields[j]
                np.savetxt(path+'/'+seq+'_'+ep[par[k]]+'-excess.txt', dfm.values, fmt='%10.5f')
                shutil.move(os.path.join(path, seq+'_'+ep[par[k]]+'-excess.txt'), 
                            os.path.join(pathbeta+'/'+ep[par[k]]+'_excess', seq+'_'+ep[par[k]]+'-excess.txt'))
                del dfm,seq
            del df

# Difference (from IdealDNA)

In [ ]:
incons  = ['oring','pcirc']
ffields = ['kabsch','olson']

par     = ['Roll','Twist','Bend','Energy','Dcenter']
ep      = {'Roll':'roll','Twist':'twist','Bend':'bend','Energy':'energy','Dcenter':'radius'}

inseqs = ['col'+str(i).zfill(2) for i in range(1,10)]

for i in range(0, len(incons)):
    for j in range(0, len(ffields)):
        for k in range(0, len(par)):
            df = pd.read_csv(path_dp+'/'+incons[i]+'_'+ffields[j]+'_'+par[k], index_col=0)
            df = df.reset_index()
            for m in range(0, len(inseqs)):
                dfm = df['delta*_'+inseqs[m]].copy()      
                dfm.loc[150] = 0.00
                for x in range(0, len(dfm)):
                    A = dfm.loc[x]
                    B = (2*(Ncirc+1)-x)-1
                    dfm.loc[B] = A
                dfm = dfm.sort_index()
                
                seq = inseqs[m]+'_'+incons[i]+'_std_'+ffields[j]
                np.savetxt(path+'/'+seq+'_'+ep[par[k]]+'-diff.txt', dfm.values, fmt='%10.5f')
                shutil.move(os.path.join(path, seq+'_'+ep[par[k]]+'-diff.txt'), 
                            os.path.join(pathbeta+'/'+ep[par[k]]+'_diff', seq+'_'+ep[par[k]]+'-diff.txt'))
                del dfm,seq
            del df